In [2]:
 # metapackage of all tidyverse packages
library(tidyverse)
# library for manipulating scales
library(scales)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor




In [3]:
# Gathering the data
price_data <- read.csv("/kaggle/input/sc-ship-price-data/sc_ship_prices_322_vs_323.csv")

In [4]:
# Taking a look at the data 
head(price_data)

# Checking all the colnames
colnames(price_data)

,location,ship,price_322,price_323
,<chr>,<chr>,<int>,<int>
1,Lorville,Hoverquad,88000,56700
2,Lorville,Mule,15000,68040
3,Lorville,Greycat ROC,172000,103950
4,Lorville,X1,10584,10584
5,Lorville,X1 Velocity,11007,11007
6,Lorville,Tumbril Cyclone,28086,110170


[1] "location"  "ship"      "price_322" "price_323"

We can get a glimpse of the content in the table above.

In [5]:
# This returns all the ships sold at Lorville.
subset(price_data, location == "Lorville")

,location,ship,price_322,price_323
,<chr>,<chr>,<int>,<int>
1,Lorville,Hoverquad,88000,56700
2,Lorville,Mule,15000,68040
3,Lorville,Greycat ROC,172000,103950
4,Lorville,X1,10584,10584
5,Lorville,X1 Velocity,11007,11007
6,Lorville,Tumbril Cyclone,28086,110170
7,Lorville,Ursa Rover,70267,120186
8,Lorville,Tumbril Cyclone RC,28086,130201
9,Lorville,Tumbril Cyclone RN,28086,130201


In [6]:
# Ordering the ship price in descending order.
ship_price_ordered <- price_data %>% arrange(desc(price_322))
print(ship_price_ordered)

    location                      ship price_322 price_323
1   Lorville                  890 Jump  32294500  65365200
2     Area18                  890 Jump  32294500  65265200
3     Area18                   Carrack  26657500  34398000
4   Lorville                    Hull C  15750000  16537500
5   Lorville                 Reclaimer  15126400  31752000
6   Lorville                Hammerhead  12459900  47958752
7     Area18              600i Touring   9894000  24938550
8     Area18                      600i   9475100  27231750
9     Area18               San'tok.yai   9355500   9355500
10  Lorville                  Redeemer   8675500  17199000
11  Lorville                 Starfarer   6651500  13230000
12  Lorville                      400i   6352700  11466000
13    Area18                      400i   6352700  11466000
14  Lorville          Starfarer Gemini   6191500  14994000
15  Lorville     Constellation Phoenix   5658800  14817600
16    Orison    A2 Hercules Starlifter   5525000  442968

In [164]:
#Obtaining the highest ship price from version 3.22 
df_max_price_322 <- price_data %>%
group_by(location) %>%
summarize(price_data = max(price_322, na.rm = TRUE))

#Obtaining the highest ship price from version 3.23

df_max_price_323 <- price_data %>%
group_by(location) %>%
summarize(price_data = max(price_323, na.rm = TRUE))

# Finding the most expensive and cheapest ships across both versions to scale the graph.
max_price <- max(price_data$price_322, price_data$price_323)
min_price <- min(price_data$price_322, price_data$price_323)

per_location_price_increase <- ggplot(mapping = aes(x = location)) +
  geom_bar(data = df_max_price_322, aes(y = price_data, fill = "price_322"), width = 0.5, stat = 'identity') +
  geom_bar(data = df_max_price_323, aes(y = price_data, fill = "price_323"), width = 0.5, stat = 'identity', alpha = 0.5) +
  geom_text(data = df_max_price_323, aes(y = price_data, label = sprintf("%.2fM", price_data / 1000000)), vjust = -0.5, hjust = 1.2, size = 3.5, color = "black") +
  geom_text(data = df_max_price_322, aes(y = price_data, label = sprintf("%.2fM", price_data / 1000000)), vjust = -0.5, hjust = 1.2, size = 3.5, color = "black") +
  labs(title = "Most expensive buyable ship by location \n 3.22 vs 3.23",
       x = "Location",
       y = "Price (in millions)",
      fill = "Versions") +
scale_fill_brewer(type = "qual", labels = c("price_322" = "Version 3.22", "price_323" = "Version 3.23")) +
  theme_light() +
  scale_y_continuous(labels = unit_format(unit = "M")) +
  theme(plot.title = element_text(hjust = 0.5)) +
coord_flip()

# saving the plot
ggsave("per_location_price_increase.png",
       plot = per_location_price_increase)

Saving 12.5 x 6.67 in image


In [26]:
# Finding the ship which had the biggest price increase from 3.22 to 3.23 

# Calculate price change
ship_price_delta <- price_data %>%
  arrange(ship) %>%
  mutate(price_increase = price_323 - price_322)

max_price_delta <- ship_price_delta %>%
  slice(which.max(price_increase))  # Retrieve the row with the maximum price increase

# Selecting the top 5 vehicles to avoid graph clutter

top_5 <- ship_price_delta %>%
  arrange(desc(price_increase)) %>%
    head(5)

# Calculating the percentage of the price increase on the top 5 vehicles

top_5_percent <- top_5 %>%
  mutate(percentage_increase = ((price_323 - top_5$price_322) / top_5$price_322 * 100) / 100)  %>%
arrange(desc(percentage_increase))

  location                   ship price_322 price_323 price_increase
1   Orison A2 Hercules Starlifter   5525000  44296876       38771876
2   Orison M2 Hercules Starlifter   5225000  29484000       24259000
3 Lorville             Hammerhead  12459900  47958752       35498852
4 Lorville               890 Jump  32294500  65365200       33070700
5   Area18               890 Jump  32294500  65265200       32970700
  percentage_increase
1            7.017534
2            4.642871
3            2.849048
4            1.024035
5            1.020939


In [62]:
# Finding the max percentage increase
max_percent_delta <- max(top_5_percent$percentage_increase)


# Plotting the graph
top_5_plot <- ggplot(top_5_percent, aes(x = location, y = percentage_increase, fill = percentage_increase)) +
  geom_bar(stat = "identity", width = 0.4) +
geom_text(aes(label = sprintf("%.2f%%", percentage_increase * 100)), vjust = -0.5) + # Add labels at the top of each bar
  labs(title = "Top 5 ship price increases, relative to 3.22",
       x = "Location",
       y = "Price Increase (%)") +
  theme(plot.title = element_text(hjust = 0.5)) +
scale_y_continuous(labels = scales::percent_format(accuracy = 0.1),
                  expand = expansion(mult = c(0, 0.1))) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) + 
facet_wrap(~ship, scales = "free") +  # Facet by dealer to show each instance separately
 guides(fill = "none")  # Remove the legend for fill (price_increase)

# saving the plot
ggsave("top_5_price_increase.png",
       plot = top_5_plot)

Saving 12.5 x 6.67 in image
